# COVID-19 Overview
> Tracking coronavirus total cases, deaths and new cases by country. Additionally, a detailed view is provided for the US(by state), Europe

- comments: true
- author: Pratap Vardhan
- categories: [overview, interactive]
- image: images/covid-overview.png
- permalink: /covid-overview/
- hide: false

In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [2]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [3]:
#hide

# FETCH
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [4]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()

In [5]:
#hide
df_table = (pd.DataFrame(dict(Cases=dfc_cases, Deaths=dfc_deaths, PCases=dfp_cases, PDeaths=dfp_deaths))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

Country/Region     Cases  Deaths    PCases  PDeaths  Cases (+)  \
0               US  33747513  605906  33679433   604943      68080   
1            India  30663665  404211  30458251   400312     205414   
2           Brazil  18855015  526892  18622304   520095     232711   
3           France   5852599  111420   5839967   111298      12632   
4           Russia   5591030  137005   5472722   133633     118308   
5           Turkey   5454763   49996   5430940    49774      23823   
6   United Kingdom   4975903  128532   4844944   128426     130959   
7        Argentina   4574340   96983   4491551    94772      82789   
8         Colombia   4402582  110019   4269297   107137     133285   
9            Italy   4264704  127704   4260788   127587       3916   
10           Spain   3880612   80952   3821305    80883      59307   
11         Germany   3739575   91118   3736959    91014       2616   
12            Iran   3286923   85095   3218860    84389      68063   
13          Poland   2880403   75095   2880010    75044        393   
14          Mexico   2549862  233958   2525350   233248      24512   

    Deaths (+)  Fatality Rate      Continent  
0          963            1.8  North America  
1         3899            1.3           Asia  
2         6797            2.8  South America  
3          122            1.9         Europe  
4         3372            2.5         Europe  
5          222            0.9           Asia  
6          106            2.6         Europe  
7         2211            2.1  South America  
8         2882            2.5  South America  
9          117            3.0         Europe  
10          69            2.1         Europe  
11         104            2.4         Europe  
12         706            2.6           Asia  
13          51            2.6         Europe  
14         710            9.2  North America

In [6]:
#hide
#delete problematic countries from table
df_table = df_table[~df_table['Country/Region'].isin(['Cape Verde', 'Cruise Ship', 'Kosovo'])]

In [7]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Cases (+)', 'Deaths (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary

{'updated': Timestamp('2021-07-06 00:00:00'),
 'since': Timestamp('2021-07-01 00:00:00'),
 'Cases': 184504193,
 'Deaths': 3990895,
 'Cases (+)': 1970527,
 'Deaths (+)': 37529,
 'China Cases': 103948,
 'China Deaths': 4848,
 'China Cases (+)': 157,
 'China Deaths (+)': 1,
 'US Cases': 33747513,
 'US Deaths': 605906,
 'US Cases (+)': 68080,
 'US Deaths (+)': 963,
 'EU Cases': 48431115,
 'EU Deaths': 1106736,
 'EU Cases (+)': 376754,
 'EU Deaths (+)': 4557}

In [8]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  \
Country/Region                                                                  
Afghanistan           0        0        0        0        0        0        0   
Albania               0        0        0        0        0        0        0   
Algeria               0        0        0        0        0        0        0   
Andorra               0        0        0        0        0        0        0   
Angola                0        0        0        0        0        0        0   

                1/29/20  1/30/20  1/31/20  ...  6/27/21  6/28/21  6/29/21  \
Country/Region                             ...                              
Afghanistan           0        0        0  ...        0     4159     1407   
Albania               0        0        0  ...        3        1        1   
Algeria               0        0        0  ...      352      375      389   
Andorra               0        0        0  ...        0        0       18   
Angola                0        0        0  ...       28       57       69   

                6/30/21  7/1/21  7/2/21  7/3/21  7/4/21  7/5/21  7/6/21  
Country/Region                                                           
Afghanistan        1501    1557       0       0       0    5721    1527  
Albania               7       2       3       8       1       2       7  
Algeria             397     449     475     457     464     495     481  
Andorra              11       7       0       0       0       0      73  
Angola              167     116     124      83      58      70      75  

[5 rows x 532 columns]

In [9]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table,  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

Visualizations by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Source: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [notebook](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-overview.ipynb), [orignal interactive](https://gramener.com/enumter/covid19/)